In [1]:
import pandas as pd
import numpy as np
import datasets

In [34]:
dset = datasets.load_dataset('indonlp/nusa_t2t', 'train_t2t_data_v2.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/scahyawijaya/.cache/huggingface/datasets/indonlp___csv/indonlp--nusa_t2t-9f34c86968dcf413/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [86]:
dset_v2 = dset['train'].filter(lambda ex: ex["dataset_name"] == 'nusa_t2t_v2')
df = dset_v2.to_pandas()
df.groupby('subset_name').size()

Filter:   0%|          | 0/41171900 [00:00<?, ? examples/s]

In [100]:
idf = pd.read_csv('v2_data/identity_prompt.csv')
idf = idf.rename({'Prompt': 'input', 'Answer': 'output'}, axis='columns')
idf['dataset_name'] = 'nusa_t2t_v2'
idf['subset_name'] = 'identity_prompt'
idf['prompt_id'] = 'identity_prompt'
idf['template_name'] = 'identity_prompt'
idf['dataset_key'] = 'identity_prompt'

In [105]:
ddf = pd.read_csv('v2_data/databricks-dolly-15k_mt_nllb-200-3.3B.csv')
ddf['subset_name'] = 'dolly'

In [108]:
v2_df = pd.concat([df,idf, ddf])
v2_df.groupby('subset_name').size().sort_values(ascending=False) / len(v2_df)

In [279]:
v2_dist_df = v2_df.groupby('subset_name').size().sort_values(ascending=False) / len(v2_df)
sr_df = (v2_dist ** 0.5) / (v2_dist ** 0.5).sum()
adj_sr_df= sr_df / v2_dist_df
multiplier_df = np.ceil(adj_sr_df)

In [283]:
dfs = []
for name, multiplier in multiplier_df.items():
    if multiplier < 1:
        multiplier = 1
    if name == 'identity_prompt':
        multiplier = 1000
    xdf = v2_df.loc[v2_df['subset_name'] == name]
    dfs += [xdf for _ in range(int(multiplier))]
adj_v2_df = pd.concat(dfs)

In [291]:
(adj_v2_df.groupby('subset_name').size() / len(adj_v2_df)).sort_values(ascending=False)

subset_name
wikipedia_id         0.368688
wikipedia_min        0.173948
wikipedia_ms         0.127253
wikipedia_jv         0.068407
wikipedia_su         0.056557
indo_puisi           0.035102
wikihow              0.029720
wikipedia_ban        0.028180
wikipedia_map-bms    0.023344
wikipedia_gor        0.022625
wikipedia_bjn        0.020260
wikipedia_ace        0.012175
dolly                0.010031
identity_prompt      0.007593
wikipedia_nia        0.007539
wikipedia_tet        0.006617
wikipedia_bug        0.001961
dtype: float64